# Hypothesis test example

We believe that the proportion of publications wronlgy categorized is different that 20%

*   **Population**: Items published under Cellphone´s Category
*   **Parameter of interest**: p

Null Hypothesis: 
>$H_{0}: p=0.2$

Alternative Hypothesis:
>$H_{a}: p\neq0.2$

Where $p$ is the population proportion of items wrongly categorized.

Significance Level: 
>$\alpha = 0.05$


In [37]:
p_null = 0.2
alpha = 0.05 

## Load the dataset

In [56]:
import pandas as pd

def load_random_sample_df(csv_path):
    df = pd.read_csv(csv_path)
    return df.sample(frac=1).reset_index(drop=True)

df = load_random_sample_df("all_data.csv")
df.head()


,ITE_ITEM_TITLE,CATEGORY
0,iPhone 8 Plus 128 Gb En Caja Sellada + Templad...,CELLPHONES
1,Celular Motorola Z2 Play Power Edition,CELLPHONES
2,Celular Nextel I355 I355is Anti Explosivo Film...,CELLPHONES
3,LG Lote De Celulares Permuto Por Un Smartphone,CELLPHONES
4,Samsung S6 Edge Libre Ya No Lo Tengo M L Robaron,CELLPHONES


## Build and Tag a Simple Random Sample

In [57]:
import requests
import ipywidgets as widgets

In [58]:
items_per_class = 10
tags = {"Wrong":'danger', "OK":"success", "Skip":""}
tag_count = {tag:0 for tag in tags.keys()}
df['Tag'] = ""

In [59]:
item_index = 0

def end_analysis(tag_count):
    for key, val in tag_count.items():
        if key != "Skip" and val < items_per_class:
            return False
    return True

def tag_button_clicked(b):
    # Display the message within the output widget.
    global item_index
    with output:
        if b.description != "Skip":
            df.at[item_index, 'Tag'] = b.description
            tag_count[b.description] += 1 

    item_index += 1
    if end_analysis(tag_count):
        print("Well Done, we have enough items for the analysis!")

    next_item = df.iloc[item_index]
    title_box.value = next_item['ITE_ITEM_TITLE']

print("Is the following Publication a CELLPHONE?")
item = df.iloc[item_index]
title_box = widgets.Label(value=item["ITE_ITEM_TITLE"])
display(title_box)
output = widgets.Output()

tag_buttons=[]
for tag in tags.keys():
    button = widgets.Button(description=tag, button_style=tags[tag])
    button.on_click(tag_button_clicked)
    tag_buttons.append(button)
display(widgets.HBox(tag_buttons))

display(output)

Is the following Publication a CELLPHONE?


Label(value='iPhone 8 Plus 128 Gb En Caja Sellada + Templado ¡ En Oferta!')

Output()

Well Done, we have enough items for the analysis!


## Do the calculations

In [60]:
import json
import statsmodels.api as sm
import numpy as np
import pandas as pd
import scipy.stats.distributions as dist

df_analyzed = df[df['Tag'] != ''].copy()
n = df_analyzed['Tag'].count()
print(f"Sample size n = { n }")
print(json.dumps(tag_count))

df_analyzed.head()

Sample size n = 24
{"Wrong": 10, "OK": 14, "Skip": 0}


,ITE_ITEM_TITLE,CATEGORY,Tag
0,iPhone 8 Plus 128 Gb En Caja Sellada + Templad...,CELLPHONES,OK
1,Celular Motorola Z2 Play Power Edition,CELLPHONES,OK
2,Celular Nextel I355 I355is Anti Explosivo Film...,CELLPHONES,OK
3,LG Lote De Celulares Permuto Por Un Smartphone,CELLPHONES,OK
4,Samsung S6 Edge Libre Ya No Lo Tengo M L Robaron,CELLPHONES,OK


In [61]:
# You should customize this code for you p
def calculate_best_estimate():
    return tag_count["Wrong"] / (tag_count["OK"] + tag_count["Wrong"])

In [62]:
p_hat = calculate_best_estimate()
print(f"Best Estimate: {p_hat}")

z, p_val = sm.stats.proportions_ztest(p_hat * n, n, p_null, alternative='two-sided')
print(f"z = {z}")
print(f"p_val = {p_val}")

Best Estimate: 0.4166666666666667
z = 2.1530045451481454
p_val = 0.031318321330661814


## Conclusion

In [64]:
if p_val < alpha:
    print(f"We have enough evidence to conclude that the proportion of wrongly categorized publications is different than {p_null}")

We have enough evidence to conclude that the proportion of wrongly categorized publications is different than 0.2
